In [4]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt 
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The follow

In [ ]:
#Get the Data
df_paid = pd.read_csv("pp-monthly-update-new-version.csv")

# Assign columns
df_paid.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [ ]:
# Format the dates
df_paid['Date_Transfer'] = df_paid['Date_Transfer'].apply(pd.to_datetime)

# Delete all transactions which were done before 2017
df_paid.drop(df_paid[df_paid.Date_Transfer.dt.year < 2017].index, inplace=True)

# Sort by Date of Sale
df_paid.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

Scope dataset to London

In [ ]:
df_london = df_paid.query("Town_City == 'LONDON'")

# create a list for streets in LONDON
streets = df_london['Street'].unique().tolist()

Calculate average price on street

In [ ]:
df_avg_price = df_london.groupby(['Street'])['Price'].mean().reset_index()
df_avg_price.columns = ['Street', 'Avg_Price']

Create dataset that meets the budget

In [ ]:
#this should be changed according to budget. This will search based on avg price
df_budget = df_avg_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2250000)")

In [14]:
df_budget

,Street,Avg_Price
1005,DEVONIA ROAD,2222500.0
1031,DORA ROAD,2250000.0
1175,ELMS CRESCENT,2200000.0
1616,HAVANNAH STREET,2217309.0
1848,JUNCTION PLACE,2200000.0
2326,MOORE PARK ROAD,2200000.0
2367,MUNDANIA ROAD,2200000.0
2441,NORRICE LEA,2250000.0
3004,SOUTH END,2200000.0
3291,THORNHILL SQUARE,2200000.0


In [1]:
#get coordinates of the sstreets with this script:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim()
# location = geolocator.geocode("TRINITY CRESCENT london")
# print(location.address)
# print((location.latitude, location.longitude))

#save them in a csv file

In [20]:
#create dataframe 
london_coord = pd.read_csv("Streets coordinates.csv")

# drop the word 'LONDON'
london_coord['Street'] = london_coord['Street'].str.replace('LONDON', '')
london_coord['Street'] = london_coord['Street'].str.replace('london', '')

In [21]:
london_coord

,Street,Latitude,Longitude
0,DEVONIA ROAD,51.534655,-0.100575
1,DORA ROAD,51.429641,-0.201156
2,ELMS CRESCENT,51.452552,-0.138582
3,HAVANNAH STREET,51.499633,-0.022975
4,JUNCTION PLACE,51.517843,-0.171002
5,MOORE PARK ROAD,51.478914,-0.192600
6,MUNDANIA ROAD,51.453403,-0.057098
7,NORRICE LEA,51.580362,-0.172926
8,SOUTH END,51.428603,-0.013172
9,THORNHILL SQUARE,51.540025,-0.115177


create one dataset with coordinates by joining 2 

In [24]:
df_all = pd.merge(london_coord, df_budget, on=['Street'], how='inner')

In [25]:
df_all

,Street,Latitude,Longitude,Avg_Price
0,DORA ROAD,51.429641,-0.201156,2250000.0
1,ELMS CRESCENT,51.452552,-0.138582,2200000.0
2,HAVANNAH STREET,51.499633,-0.022975,2217309.0
3,JUNCTION PLACE,51.517843,-0.171002,2200000.0
4,MOORE PARK ROAD,51.478914,-0.192600,2200000.0
5,MUNDANIA ROAD,51.453403,-0.057098,2200000.0
6,NORRICE LEA,51.580362,-0.172926,2250000.0
7,SOUTH END,51.428603,-0.013172,2200000.0
8,THORNHILL SQUARE,51.540025,-0.115177,2200000.0
9,TRINITY CRESCENT,51.438171,-0.159411,2200000.0


In [22]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


create map of london

In [27]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers
for lat, lng, price, street in zip(df_all['Latitude'], df_all['Longitude'], df_all['Avg_Price'], df_all['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Explore with Foursquare to find interested venues

In [ ]:

CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '*******' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



# Use this function to explore the streets
# def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
#     venues_list=[]
#     for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
#         # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
            
#         # make the GET request
#         results = requests.get(url).json()["response"]['groups'][0]['items']
        
#         # return only relevant information for each nearby venue
#         venues_list.append([(
#             name, 
#             lat, 
#             lng, 
#             v['venue']['name'], 
#             v['venue']['location']['lat'], 
#             v['venue']['location']['lng'],  
#             v['venue']['categories'][0]['name']) for v in results])

#     nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#     nearby_venues.columns = ['Street', 
#                   'Street Latitude', 
#                   'Street Longitude', 
#                   'Venue', 
#                   'Venue Latitude', 
#                   'Venue Longitude', 
#                   'Venue Category']
    
#     return(nearby_venues)